<a href="https://colab.research.google.com/github/NicoPozio/WildfireDetectionDL/blob/main/notebook/Classifier_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/NicoPozio/WildfireDetectionDL.git

Cloning into 'WildfireDetectionDL'...
remote: Enumerating objects: 212, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 212 (delta 8), reused 3 (delta 0), pack-reused 188 (from 1)
Receiving objects: 100% (212/212), 139.10 KiB | 11.59 MiB/s, done.
Resolving deltas: 100% (103/103), done.


In [2]:
import sys

repo_path = '/content/WildfireDetectionDL'
if repo_path not in sys.path:
    sys.path.append(repo_path)

In [12]:
# Cell: Update Code
import os

REPO_PATH = "/content/WildfireDetectionDL"

print("Syncing with GitHub...")
!cd {REPO_PATH} && git pull

print("Code updated. You can now run the Sweep or Train cell immediately.")

Syncing with GitHub...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 2), reused 0 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 1.07 KiB | 1.07 MiB/s, done.
From https://github.com/NicoPozio/WildfireDetectionDL
   aca0137..4657e90  main       -> origin/main
Updating aca0137..4657e90
Fast-forward
 conf/config.yaml | 1 +
 1 file changed, 1 insertion(+)
Code updated. You can now run the Sweep or Train cell immediately.


In [4]:
#Install Project Dependencies
#hydra-core: Configuration management
#wandb: Experiment tracking
#omegaconf: Dict handling for Hydra
!pip install -q hydra-core wandb omegaconf


from google.colab import drive
import os

if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')
    print("Google Drive Mounted")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 5.5 MB/s eta 0:00:00
Mounted at /content/drive
Google Drive Mounted


In [5]:
import wandb
from google.colab import userdata
import os

try:
    #Fetch key from Colab Secrets
    api_key = userdata.get('WANDB_API_KEY')

    #Set as Environment Variable
    #This ensures Hydra and subprocesses can find it automatically
    os.environ["WANDB_API_KEY"] = api_key

    wandb.login()
    print("Logged in to WandB")

except Exception as e:
    print(f"Authentication Failed: {e}")
    print("Action Required: Go to the 'Secrets' tab (Key icon) on the left.")
    print("Add a new secret named 'WANDB_API_KEY' with your key from https://wandb.ai/authorize")

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Currently logged in as: pozioniccolo (pozioniccolo-sapienza-universit-di-roma) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Logged in to WandB


First we download the dataset, we check if the user has the data in its google drive, if not it's downloaded from a public link

In [7]:
import os
import gdown
from src.utils import extract_zip

#Destination on Colab
local_root = "/content/data"
os.makedirs(local_root, exist_ok=True)
!rm -rf {local_root}/*

#Paths on drive (in case the CyclaGAN notebook is exectued)
drive_dataset_path = "/content/drive/MyDrive/Wildfire_Project/dataset.zip"
drive_synthetic_path = "/content/drive/MyDrive/Wildfire_Project/synthetic_wildfire_2k.zip"

#Paths on Colab
target_dataset_zip = os.path.join(local_root, "dataset.zip")
target_synthetic_zip = os.path.join(local_root, "synthetic_wildfire_2k.zip")



#Check if the file are present
if os.path.isfile(drive_dataset_path) and os.path.isfile(drive_synthetic_path):
    !cp "{drive_dataset_path}" "{target_dataset_zip}"
    !cp "{drive_synthetic_path}" "{target_synthetic_zip}"

else:

    DATASET_ID = "17KPBVodZkmBYqz7252mDk6hfY55eMU-Q"
    SYNTHETIC_ID = "1KyI09FDCAkLp1BYO-VgD7zRMrtrl3anK"

    gdown.download(id=DATASET_ID, output=target_dataset_zip, quiet=False)

    gdown.download(id=SYNTHETIC_ID, output=target_synthetic_zip, quiet=False)


if os.path.exists(target_dataset_zip) and os.path.exists(target_synthetic_zip):
    if os.path.getsize(target_dataset_zip) > 0:
        extract_zip(target_dataset_zip, local_root)
    if os.path.getsize(target_synthetic_zip) > 0:
        extract_zip(target_synthetic_zip, local_root)
    print("Data Ready")

Extracting dataset.zip to /content/data...


Unzipping: 100%|██████████| 42860/42860 [00:11<00:00, 3693.59files/s]


 -> Success: Extracted to /content/data
Extracting synthetic_wildfire_2k.zip to /content/data...


Unzipping: 100%|██████████| 2003/2003 [00:01<00:00, 1402.05files/s]

 -> Success: Extracted to /content/data
Data Ready


In [8]:
import os
from PIL import Image
from tqdm import tqdm

# Define your data root
DATA_ROOT = "/content/data"

print(f"Scanning {DATA_ROOT} for corrupt images...")

corrupt_count = 0
for root, dirs, files in os.walk(DATA_ROOT):
    for filename in files:
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            file_path = os.path.join(root, filename)
            try:
                # Try to fully load the image bytes
                with Image.open(file_path) as img:
                    img.load()
            except OSError:
                print(f"Found corrupt image: {file_path}")
                try:
                    os.remove(file_path)
                    print(f"Deleted {filename}")
                    corrupt_count += 1
                except:
                    print(f"Could not delete {filename}")

print(f"\nScan Complete. Removed {corrupt_count} corrupt files.")

Scanning /content/data for corrupt images...
Found corrupt image: /content/data/dataset/test/wildfire/-73.15884,46.38819.jpg
Deleted -73.15884,46.38819.jpg
Found corrupt image: /content/data/dataset/train/nowildfire/-114.152378,51.027198.jpg
Deleted -114.152378,51.027198.jpg

Scan Complete. Removed 2 corrupt files.


In [14]:
import subprocess
import os

# 1. Define Paths Constants
REPO_ROOT = "/content/WildfireDetectionDL"
SWEEP_CONFIG_PATH = os.path.join(REPO_ROOT, "conf", "sweep.yaml")

# 2. Validation
if not os.path.exists(SWEEP_CONFIG_PATH):
    print(f"CRITICAL ERROR: Could not find sweep.yaml at {SWEEP_CONFIG_PATH}")
else:
    print(f"Registering Sweep from {SWEEP_CONFIG_PATH}")

    # 3. Register the sweep
    result = subprocess.run(
        ["wandb", "sweep", SWEEP_CONFIG_PATH],
        capture_output=True,
        text=True,
        # CORRECTION: cwd must be the directory, not the file
        cwd=REPO_ROOT
    )

    output_text = result.stderr + result.stdout
    print("Raw Output:", output_text)

    # 4. Extract Sweep ID
    sweep_id = None
    for line in output_text.split('\n'):
        if "wandb agent" in line:
            parts = line.strip().split("wandb agent ")
            if len(parts) > 1:
                sweep_id = parts[-1].strip()
                break

    # 5. Launch the Agent
    if sweep_id:
        print(f"\nSUCCESS: Detected Sweep ID: {sweep_id}")
        print("Starting Agent... (This will run multiple experiments)")

        # CORRECTION: We use the explicit REPO_ROOT variable defined at the top
        !cd {REPO_ROOT} && wandb agent {sweep_id} --count 20
    else:
        print("\nERROR: Could not find Sweep ID.")

Registering Sweep from /content/WildfireDetectionDL/conf/sweep.yaml
Raw Output: wandb: Creating sweep from: /content/WildfireDetectionDL/conf/sweep.yaml
wandb: Creating sweep with ID: gq3l2a4v
wandb: View sweep at: https://wandb.ai/pozioniccolo-sapienza-universit-di-roma/WildfireDetectionDL/sweeps/gq3l2a4v
wandb: Run sweep agent with: wandb agent pozioniccolo-sapienza-universit-di-roma/WildfireDetectionDL/gq3l2a4v


SUCCESS: Detected Sweep ID: pozioniccolo-sapienza-universit-di-roma/WildfireDetectionDL/gq3l2a4v
Starting Agent... (This will run multiple experiments)
wandb: Starting wandb agent 🕵️
2025-12-01 21:40:28,192 - wandb.wandb_agent - INFO - Running runs: []
2025-12-01 21:40:28,523 - wandb.wandb_agent - INFO - Agent received command: run
2025-12-01 21:40:28,523 - wandb.wandb_agent - INFO - Agent starting run with config:
	dataset.augmentation.rotation_degrees: 13
	dataset.params.use_synthetic: False
	model.dropout: 0.5
	model.name: resnet50
	training.batch_size: 32
	training.lear

With the sweep execution we found the best configuration for the neural networks, now we train the resnet50 using that parameters, we train using both synthetic+real, and only real

In [13]:
import subprocess
import sys
import os

#Hyperparameter Configurations

# 1. ResNet50 (Transfer Learning)
RESNET_CONFIG = [
    "model.name=resnet50",
    "model.dropout=0.5",
    "dataset.augmentation.rotation_degrees=17",
    "dataset.params.real_data_fraction=0.1",
    "training.batch_size=64",
    "training.learning_rate:0.00005836874490583941",
    "training.optimizer=adam",
    "training.weight_decay:0.00000621085260935666",
    "training.epochs=20"
]

# 2. SimpleCNN (Custom Architecture)
SIMPLE_CNN_CONFIG = [
    "model.name=simple_cnn",
    "model.dropout=0.5",
    "dataset.augmentation.rotation_degrees=4",
    "dataset.params.real_data_fraction=0.1",
    "training.batch_size=64",
    "training.learning_rate:0.00001349358355802241",
    "training.optimizer=adam",
    "training.weight_decay:0.00000823684190127468",
    "training.epochs=20"
]

# 3. EfficientNet (Modern Architecture)
EFFICIENTNET_CONFIG = [
    "model.name=efficientnet",
    "model.dropout=0.2",
    "dataset.augmentation.rotation_degrees=15",
    "dataset.params.real_data_fraction=0.1",
    "training.batch_size=32", # Smaller batch size for stability
    "training.learning_rate=0.0001",
    "training.optimizer=adam",
    "training.weight_decay=0.0",
    "training.epochs=20"
]

#Pipeline Logic

def run_training_pipeline(model_label, use_synthetic, config_list):
    run_id = f"{model_label}_{'Synthetic' if use_synthetic else 'RealOnly'}"
    print(f"STARTING TRAINING PIPELINE: {run_id}")

    #Construct Training Command
    train_cmd = [
        sys.executable, "train.py",
        f"dataset.params.use_synthetic={str(use_synthetic).lower()}",
        "wandb.project=wildfire-final-benchmark",
        f"+wandb.name={run_id}"
    ] + config_list

    #Execute Training
    try:
        process = subprocess.Popen(
            train_cmd,
            cwd="/content/WildfireDetectionDL",
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True
        )

        #Stream output to console
        for line in process.stdout:
            print(line, end="")

        process.wait()

        if process.returncode != 0:
            print(f"\nTRAINING FAILED: {run_id}")
            return False
        else:
            print("\nTraining Complete.")
            return True

    except Exception as e:
        print(f"CRITICAL ERROR: {e}")
        return False

#ResNet50
run_training_pipeline("ResNet50", False, RESNET_CONFIG)
run_training_pipeline("ResNet50", True, RESNET_CONFIG)

#SimpleCNN
run_training_pipeline("SimpleCNN", False, SIMPLE_CNN_CONFIG)
run_training_pipeline("SimpleCNN", True, SIMPLE_CNN_CONFIG)

#EfficientNet
run_training_pipeline("EfficientNet", False, EFFICIENTNET_CONFIG)
run_training_pipeline("EfficientNet", True, EFFICIENTNET_CONFIG)


STARTING EXPERIMENT: Run_A_Real_Only
   Synthetic Data Enabled: False

wandb: Currently logged in as: pozioniccolo (pozioniccolo-sapienza-universit-di-roma) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.23.0
wandb: Run data is saved locally in /content/WildfireDetectionDL/wandb/run-20251201_211945-zjjf1tpx
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run quiet-bush-1
wandb: ⭐️ View project at https://wandb.ai/pozioniccolo-sapienza-universit-di-roma/wildfire-ab-test
wandb: 🚀 View run at https://wandb.ai/pozioniccolo-sapienza-universit-di-roma/wildfire-ab-test/runs/zjjf1tpx
Training on cuda using model: resnet50
Real Data Loaded. Train Size: 30249
BASELINE MODE: Using only Real data.
Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth

  0%|          | 0.00/97.8M [00:00<?, ?B/s]
 18%|█▊        | 17.2M/97.8M [00:00<00:00, 180M

KeyboardInterrupt: 

In [ ]:
import wandb
import os
import subprocess
import pandas as pd

# CONFIGURATION
api = wandb.Api()
CURRENT_ENTITY = api.default_entity
PROJECT = "wildfire-final-benchmark" # Must match training project

# List of experiment names we expect to find
EXPERIMENTS = [
    ("ResNet50_RealOnly", "resnet50", 0.5),
    ("ResNet50_Synthetic", "resnet50", 0.5),
    ("SimpleCNN_RealOnly", "simple_cnn", 0.5),
    ("SimpleCNN_Synthetic", "simple_cnn", 0.5),
    ("EfficientNet_RealOnly", "efficientnet", 0.2),
    ("EfficientNet_Synthetic", "efficientnet", 0.2),
]

print(f"Running Analysis on Project: {CURRENT_ENTITY}/{PROJECT}")

def test_run(run_name, model_type, dropout_val):
    print(f"ANALYZING: {run_name}")

    # Find Run ID
    path = f"{CURRENT_ENTITY}/{PROJECT}"
    try:
        runs = api.runs(path, filters={"display_name": run_name})
    except Exception as e:
        print(f"   Connection Error: {e}")
        return

    if len(runs) == 0:
        print(f"   Run not found: {run_name}")
        return

    target_run = runs[0] # Get most recent
    print(f"   Found Run ID: {target_run.id}")

    #Download Weights
    try:
        local_weight_name = f"weights_{target_run.id}.pth"
        target_run.file("best_model.pth").download(replace=True, root=".")
        os.rename("best_model.pth", local_weight_name)
        print(f"   Weights downloaded: {local_weight_name}")
    except Exception as e:
        print(f"   Download Failed: {e}")
        return

    #Determine Synthetic Flag based on run name
    use_synthetic = "Synthetic" in run_name

    #Run Test Script
    cmd = [
        "python", "test.py",
        f"model.name={model_type}",
        f"model.dropout={dropout_val}",
        f"dataset.params.use_synthetic={str(use_synthetic).lower()}",
        # Batch size needs to match training memory constraints
        "training.batch_size=32",
        "wandb.mode=disabled"
    ]

    # Prepare file for test.py
    if os.path.exists("best_model.pth"):
        os.remove("best_model.pth")
    os.rename(local_weight_name, "best_model.pth")

    print("   Running Inference on Test Set...")
    # Capture output to show metrics
    proc = subprocess.run(
        cmd,
        cwd="/content/WildfireDetectionDL",
        capture_output=True,
        text=True
    )

    if proc.returncode == 0:
        print("Testing Success.")
        # Parse and print relevant metrics from stdout
        for line in proc.stdout.split('\n'):
            if any(k in line for k in ["Accuracy:", "Recall:", "F1-Score:"]):
                print(f"{line.strip()}")
    else:
        print("Testing Failed.")
        print(proc.stderr)

# --- EXECUTION LOOP ---
for exp_name, model_arch, drop_val in EXPERIMENTS:
    test_run(exp_name, model_arch, drop_val)

In [ ]:
import wandb
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# ---------------------------------------------------------
# 1. Configuration
# ---------------------------------------------------------
# Based on your logs: https://wandb.ai/pozioniccolo-sapienza-universit-di-roma/WildfireDetectionDL/sweeps/7hmucehp

# REPLACE THESE WITH YOUR EXACT DETAILS FROM THE LOGS
ENTITY = "pozioniccolo-sapienza-universit-di-roma"
PROJECT = "WildfireDetectionDL"  # <--- Correction: Case Sensitive!
SWEEP_ID = "7hmucehp"            # <--- Correction: Hardcoded from your logs to be safe

# Construct the full path
sweep_path = f"{ENTITY}/{PROJECT}/{SWEEP_ID}"
print(f"Connecting to: {sweep_path}...")

# ---------------------------------------------------------
# 2. Fetch Data
# ---------------------------------------------------------
api = wandb.Api()

try:
    sweep = api.sweep(sweep_path)
    print("✅ Connection Successful! Downloading runs...")

    # Get runs sorted by accuracy
    runs = sorted(sweep.runs, key=lambda run: run.summary.get("val_acc", 0), reverse=True)

    results = []
    for run in runs:
        # Only process runs that finished and have metrics
        if "val_acc" in run.summary:
            cfg = run.config

            # Helper to safely get nested keys
            def get_param(config_dict, key_path):
                keys = key_path.split('.')
                val = config_dict
                for k in keys:
                    if isinstance(val, dict):
                        val = val.get(k)
                    else:
                        return None
                return val

            # Extract data using the safe helper
            results.append({
                "run_id": run.id,
                "model": get_param(cfg, 'model.name'),
                "use_synthetic": get_param(cfg, 'dataset.params.use_synthetic'),
                "lr": get_param(cfg, 'training.learning_rate'),
                "batch_size": get_param(cfg, 'training.batch_size'),
                "optimizer": get_param(cfg, 'training.optimizer'),
                "val_acc": run.summary.get("val_acc")
            })

    df = pd.DataFrame(results)

    # ---------------------------------------------------------
    # 3. Analysis & Visualization
    # ---------------------------------------------------------
    if not df.empty:
        best_run = df.iloc[0]
        print("\n🏆 CHAMPION CONFIGURATION 🏆")
        print(f"Run ID:         {best_run['run_id']}")
        print(f"Model:          {best_run['model']}")
        print(f"Synthetic Data: {best_run['use_synthetic']}")
        print(f"Optimizer:      {best_run['optimizer']}")
        print(f"Learning Rate:  {best_run['lr']}")
        print(f"Val Accuracy:   {best_run['val_acc']:.2f}%")

        # Plot
        plt.figure(figsize=(10, 6))
        sns.barplot(data=df, x="model", y="val_acc", hue="use_synthetic", errorbar=None)
        plt.title("Model Performance: Real vs Synthetic Data")
        plt.ylim(80, 100) # Zoom in to the top range (since your acc is high)
        plt.ylabel("Validation Accuracy (%)")
        plt.xlabel("Model Architecture")
        plt.legend(title="Use Synthetic Data")
        plt.grid(axis='y', alpha=0.3)
        plt.show()

    else:
        print("⚠️ No finished runs found with validation accuracy.")

except Exception as e:
    print(f"❌ Error connecting to WandB: {e}")
    print("Check if Entity, Project, or Sweep ID are correct.")

Connecting to: pozioniccolo-sapienza-universit-di-roma/WildfireDetectionDL/7hmucehp...


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:



KeyboardInterrupt: 

In [ ]:
import wandb
import os

# 1. Configuration (Must match your logs exactly)
ENTITY = "pozioniccolo-sapienza-universit-di-roma"
PROJECT = "WildfireDetectionDL"

api = wandb.Api()

if 'best_run' in locals() and not df.empty:
    # Get the ID from the pandas row
    run_id = best_run['run_id']

    print(f"🏆 Preparing to test Best Model: {best_run['model']} (Run ID: {run_id})...")

    # 2. Construct the path using known variables
    # Format: entity/project/run_id
    run_path = f"{ENTITY}/{PROJECT}/{run_id}"

    print(f"⬇️ Downloading weights from: {run_path}")

    try:
        # 3. Download 'best_model.pth'
        run_obj = api.run(run_path)
        run_obj.file("best_model.pth").download(replace=True, root=".")

        print("✅ Download Complete. Weights are ready.")

        # 4. Run the Test
        print("🚀 Starting Test Script...")

        # Extract params for the command line
        use_syn = str(best_run['use_synthetic']).lower()
        batch_size = int(best_run['batch_size'])

        !python ./WildfireDetectionDL/test.py \
          model.name={best_run['model']} \
          dataset.params.use_synthetic={use_syn} \
          training.batch_size={batch_size} \
          wandb.mode="online"

    except Exception as e:
        print(f"❌ Error: {e}")

else:
    print("Cannot run test: No best run identified in the analysis step.")